In [1]:
import configparser
from datetime import datetime,timedelta
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
import sql_queries as sql
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read('aws.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config["CREDENTIALS"]['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config["CREDENTIALS"]['AWS_SECRET_ACCESS_KEY']
AWS_ACCESS_KEY=config["CREDENTIALS"]['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY=config["CREDENTIALS"]['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession.builder \
    .config("fs.s3a.awsAccessKeyId", AWS_ACCESS_KEY)\
    .config("fs.s3a.awsSecretAccessKey", AWS_SECRET_KEY)\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.3,org.postgresql:postgresql:42.2.6")\
    .getOrCreate()

In [4]:
## Source: https://www1.nyc.gov/site/nypd/stats/crime-statistics/historical.page
nycrime_df = spark.read.csv("s3a://udacity-cap-bs/NY_Crimes_2016.csv",header=True)

In [5]:
start=datetime.now()

In [6]:
nycrime_df.repartition(10).write.parquet(os.path.join("data", 'nycrime.parquet'), 'overwrite')

In [7]:
print(datetime.now()-start)

0:00:30.310535


In [8]:
airbnb_df = spark.read.csv("s3a://udacity-cap-bs/AB_NYC_2019.csv",header=True)

In [9]:
airbnb_df.write.parquet(os.path.join("data", 'airbnb.parquet'), 'overwrite')

In [10]:
## Source: https://www.kaggle.com/mathijs/weather-data-in-new-york-city-2016
clima_df = spark.read.csv("s3a://udacity-cap-bs/weather_data_nyc_centralpark_2016.csv",header=True)

In [11]:
for col in clima_df.columns:
    clima_df = clima_df.withColumnRenamed(col, col.replace(" ",""))
clima_df.write.parquet(os.path.join("data", 'clima.parquet'), 'overwrite')